In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd

# Step 1: Clean the original file
input_file = "output_trained.csv"
output_file = "output_trained_fixed.csv"

fixed_lines = []
bad_lines = []

with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

for i, line in enumerate(lines, start=1):
    line = line.strip()
    if not line:
        continue  # skip empty lines
    parts = line.split(",", 1)  # split only at the first comma
    if len(parts) == 2:
        label = parts[0].strip()
        sentence = parts[1].strip().replace('"', '""')  # escape inner quotes
        fixed_lines.append(f'{label},"{sentence}"\n')
    else:
        bad_lines.append((i, line))

# Step 2: Save the cleaned file
with open(output_file, "w", encoding="utf-8") as f:
    f.write("Label,Sentence\n")  # CSV header
    f.writelines(fixed_lines)

# Step 3: Report any issues
if bad_lines:
    print(f"⚠️ {len(bad_lines)} malformed lines found:")
    for line_num, content in bad_lines:
        print(f"Line {line_num}: {content}")
else:
    print("✅ All lines processed correctly.")

# Step 4: Read cleaned CSV
df = pd.read_csv(output_file)
print("✅ Fixed file loaded successfully.")
print("📊 Number of rows:", len(df))
df.head()


✅ All lines processed correctly.
✅ Fixed file loaded successfully.
📊 Number of rows: 8499


,Label,Sentence
0,Label,Sentence
1,neutral,आप अपने हाथ भरे होंगे।
2,neutral,कि मैंने किया। कि मैंने किया।
3,neutral,तो चलो अपने कर्तव्यों के बारे में थोड़ा बात करें।
4,surprise,मेरे कर्तव्य? ठीक है.


In [ ]:
# 📌 Step 2: Extract text and labels
texts = df['Sentence'].astype(str).tolist()
labels = df['Label'].astype(str).tolist()


In [ ]:
# 📌 Step 3: Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

In [ ]:
# 📌 Step 4: Tokenize and pad text
max_words = 8000
max_len = 30

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=max_len, padding='post')
y = tf.keras.utils.to_categorical(labels_encoded, num_classes=num_classes)

In [ ]:
# 📌 Step 5: Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 📌 Step 6: Build and compile the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=max_words, output_dim=64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, max_len))
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 30, 64)         │       512,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 30, 128)        │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_4          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 586,824 (2.24 MB)

 Trainable params: 586,824 (2.24 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 📌 Step 7: Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - accuracy: 0.4310 - loss: 1.7331 - val_accuracy: 0.4635 - val_loss: 1.5661
Epoch 2/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 9s 42ms/step - accuracy: 0.4639 - loss: 1.5296 - val_accuracy: 0.4694 - val_loss: 1.5227
Epoch 3/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 9s 42ms/step - accuracy: 0.4874 - loss: 1.4099 - val_accuracy: 0.4859 - val_loss: 1.4952
Epoch 4/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.5664 - loss: 1.2327 - val_accuracy: 0.4753 - val_loss: 1.5419
Epoch 5/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - accuracy: 0.6353 - loss: 1.0710 - val_accuracy: 0.4482 - val_loss: 1.6090
Epoch 6/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.6767 - loss: 0.9655 - val_accuracy: 0.4471 - val_loss: 1.8099
Epoch 7/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 0.7154 - loss: 0.8462 - val_accuracy: 0.4559 - val_loss: 1.9411
Epoch 8/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - accuracy: 0.7510 - loss: 0.7669 - val_

In [ ]:
def predict_emotion(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    pred = model.predict(padded)[0]
    emotion = label_encoder.inverse_transform([np.argmax(pred)])[0]
    confidence = np.max(pred)
    print(f"Prediction: {emotion.capitalize()} ({confidence:.2f} confidence)")

# 🔍 Test It
predict_emotion("कल मौसम साफ रहेगा।")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Prediction: Neutral (1.00 confidence)
